## Prepare Elasticsearch

In [1]:
from elasticsearch import Elasticsearch

In [2]:
# Route traffic to running docker 
es_client = Elasticsearch('http://localhost:9200')

In [3]:
# Verify 
es_client.info()

{'name': '8de99bb3d4ff',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'G2RUPustTUerubQ_H7xGuw',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Create index 

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [5]:
index_name = 'course_questions'

In [6]:
# Use es_client to create index
es_client.indices.create(index = index_name, 
                         body = index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course_questions'}

## Index the data 

In [7]:
import json
from tqdm.auto import tqdm

In [8]:
# Use created doc json 
# open documents in readtext mode 
try:
    with open('documents.json', 'rt') as f_in: 
        docs_raw = json.load(f_in)
except FileNotFoundError:
    print("The file documents.json was not found.")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding JSON from the file.")
    exit(1)


In [9]:
!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "12q`\tThe purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [36]:
docs_raw[0].keys()

dict_keys(['course', 'documents'])

In [11]:
# create list of dictioniaries 
documents = []

for course_dict in docs_raw: 
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "12q`\tThe purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
# Index all documents
for doc in tqdm(documents):
    es_client.index(index = index_name, 
                    body = doc)

  0%|          | 0/1024 [00:00<?, ?it/s]

## Querying the data with Elasticsearch

In [43]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", 
                                   "text", 
                                   "section"],
                        "type": "best_fields"
                    }
                },
                # this is optional to filter e.g. for specific courses 
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, 
                                body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

## Rag pipeline

In [38]:
def build_prompt(query, search_results): 

    prompt_template = """  

    You're a course teaching assistant.
    
    Answer the QUESTION based on the CONTEXT from the FAQ database.

    Use only the facts from the CONTEXT when answering the QUESTION.

    If you can not answer the QUESTION based on the CONTEXT, inform the user.  
        
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return prompt_template.format(question = query, 
                                  context = context).strip()

In [39]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{                 
            "role": "user", 
            "content": prompt
        }])

    return response.choices[0].message.content

In [40]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    print(llm(prompt))

In [41]:
from openai import OpenAI
client = OpenAI()

In [44]:
query = "How do I run Kafka?"
query = "What do we learn about AWS Lambda?"
query = "The course just started. How do I enroll?"
rag(query)

To enroll in the course, please follow these steps:

1. Visit the course page: [http://mlzoomcamp.com](http://mlzoomcamp.com/).
2. Scroll down to start going through the course materials.
3. Read everything in the cohort folder for your cohort’s year.
4. Click on the links provided and begin watching the videos.
5. You can also watch office hours from previous cohorts.
6. Visit the DTC YouTube channel, click on Playlists, and search for your course cohort (e.g., ML Zoomcamp 2023).

If you're following along in self-paced mode:
- Check out the pins and bookmarks in the course channel.
- Read the repository bookmarked in the channel and watch the video lessons.
- Use the FAQ document or search the channel for answers to common questions.
- If additional help is needed, you can tag @ZoomcampQABot for a summary.

You can access the course syllabus directly through this link: [http://mlzoomcamp.com/#syllabus](http://mlzoomcamp.com/#syllabus).

Welcome to the course!
